In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import sys
import os
import io
import srtm
from tqdm.auto import tqdm
from configobj import ConfigObj

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.utils import database
from functions.check_los import check_los
from functions.tower_los import tower_los

In [ ]:
config_path = "/home/jovyan/shared/rural_planner_refactoring/config_files/config_pe"

parser = ConfigObj(config_path)

sql_path = parser['sql_path']
country_folder = parser['country_folder']

interval = parser['path_finder_roads_cl_los_params']['interval']

table_towers = parser['transport_by_tower_params']['table_infrastructure']
table_cluster_node_map = parser['path_finder_roads_params']['table_cluster_node_map']
table_clusters = parser['clustering_params']['output_table']
table_settlements = parser['clustering_params']['table_settlements']
table_settlements_kpis = parser['path_finder_roads_cl_los_params']['table_settlements_kpis']
table_jungle = parser['path_finder_roads_cl_los_params']['table_jungle']
table_output = parser['path_finder_roads_cl_los_params']['table_output']

In [ ]:
#Find centroids in jungle areas and save in table and load cluster-node map
#Assumption: we deploy 30-m towers

query_path = sql_path + '/' + country_folder + '/' + 'path_finder_cluster_create_centroids_jungle.sql'
query_path_load = sql_path + '/' + country_folder + '/' + 'path_finder_cluster_load_centroids.sql'

with open(query_path) as file, open(query_path_load) as file_load, database(parser) as db:       
    query = file.read()
    query_formatted = query.format(schema = schema,
                               table_settlements = table_settlements,
                               table_towers = table_towers,
                               table_jungle = table_jungle,
                               table_settlements_kpis = table_settlements_kpis,
                               table_clusters = table_clusters,
                               table_cluster_node_map = table_cluster_node_map)
    db.execute(query_formatted)   
    
    query_load = file_load.read()
    query_formatted = query.format(schema = schema,
                               table_settlements = table_settlements,
                               table_towers = table_towers,
                               table_jungle = table_jungle,
                               table_cluster_node_map = table_cluster_node_map)
    df_nodes = pd.read_sql_query(query_formatted, db)
    
df_nodes['node_id'] = df_nodes['node_id'].astype(int)

In [ ]:
# calculate line of sight for each centroid with its closest road node
tqdm.pandas()
elevation_data = srtm.get_data()
df_nodes = df_nodes.join(df_nodes.progress_apply(towers_los, elevation_data=elevation_data, interval = interval, axis=1))

In [ ]:
#Export and modify types so they are geometries

query_path_load = sql_path + '/' + country_folder + '/' + 'path_finder_cluster_cast_geoms.sql'

with open(query_path) as file, database(parser) as db:
    df_nodes.to_sql(output_table, con=db, if_exists = 'replace', schema = schema, index = False)
    query = file.read()
    query_formatted = query.format(schema = schema,
                               table_settlements = table_settlements,
                               table_towers = table_towers,
                               table_jungle = table_jungle,
                               table_settlements_kpis = table_settlements_kpis,
                               table_clusters = table_clusters,
                               table_cluster_node_map = table_cluster_node_map)
    db.execute(query_formatted)   
    
    query_load = file_load.read()
    query_formatted = query.format(schema = schema,
                                  table_output = table_output)
    db.execute(query_formatted)